# Créer son propre dataset OCR

Ici, vous devez vous même créer les données de reconnaissance des caractères manuscrits. 
L'idée n'est pas de construire un classifieur très compliqué, mais de bien comrendre l'importance du prétraitement dans la performance de la classification. 

Il y a 2 tâches à réaliser : 
- extraire et préparer les données brutes à partir de fichiers images (des planches de nombres manuscrits)
- voir quels effets améliorent la reconnaissance. Parmis les effets à tester, on a : la couleur (RGB, greyscale ou noir et blanc), la résolution de l'image, l'ajout de flou, l'ajout de netteté, la suppression du bruit (filtre median), ...

Ci dessous, quelques fonctions utiles pour manipuler les images avec PIL.
https://pillow.readthedocs.io/en/4.0.x/reference/Image.html

In [645]:
from PIL import Image, ImageFilter 
import numpy as np
from PIL.ImageFilter import (GaussianBlur, BLUR, CONTOUR, DETAIL, EDGE_ENHANCE, EDGE_ENHANCE_MORE,
    EMBOSS, FIND_EDGES, SMOOTH, SMOOTH_MORE, SHARPEN)
import collections
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display 

In [646]:
im = Image.open("p001.bmp")

In [647]:
#im.show()

# Dataset

In [648]:
import os

valuesList = list()
labelsList = list()

def cropGrille(xinit, y, dx, dy, margex, margey, file, show=False):
    x=xinit
    im = Image.open(file)
    #im = im.filter(GaussianBlur(radius=10))
    im = im[120:220]    
    plt(im)
    
    for j in range(10):
        x=xinit
        for i in range(10):
            #box = (x, y, x+dx, y+dy)    
            box = (x+margex, y+margey, x+dx-margex, y+dy-margey)
            region = im.crop(box)
                        
            region = region.convert('L')
            tab = np.array(region)
            tab = tab.reshape((region.size[0] * region.size[1]),)
            valuesList.append(tab)
            if( file == "p010.bmp"):
                labelsList.append((j+1)%10)
            else :
                labelsList.append(j)    
            x+=dx  
        y=y+dy 


#for i in range(len(values)):
 #   print(values[i])


cropGrille(144, 221, 94, 109, 8, 12, "p001.bmp")
cropGrille(142, 215, 94, 109, 8, 12, "p002.bmp")
cropGrille(148, 225, 94, 109, 8, 12, "p003.bmp")
cropGrille(148, 225, 94, 109, 8, 12, "p005.bmp")

In [649]:
#dataset non contrarié
cropGrille(150, 225, 94, 109, 10, 14, "p004.bmp")
cropGrille(149, 225, 94, 109, 10, 14, "p006.bmp")
cropGrille(152, 225, 94, 109, 10, 14, "p007.bmp")
cropGrille(153, 225, 94, 109, 10, 14, "p008.bmp")
cropGrille(152, 225, 94, 109, 10, 14, "p009.bmp")
cropGrille(152, 225, 94, 109, 10, 14, "p010.bmp")

valuesArray = np.array(valuesList)
labelsArray = np.array(labelsList)


TypeError: 'BmpImageFile' object has no attribute '__getitem__'

### Exercice 1 Dataset
Après avoir vérifié que le cropping est satisfaisant, construisez le dataset des caractères non contrariés. 

- x doit désigner un ensemble de 1D-tableaux numpy, chaque tableau consignant les pixels d'une image
- y désigne les classes (labels).

Remarque :
- sur p010.bmp, les 0 sont au début de la feuille. 
- Sans redimensionnement, les shapes de x et y doivent être (600, 1596) et (600,)


In [ ]:
print(collections.Counter(labelsArray))

### Exercice 3 Construction du classifieur 

Testez les classifieurs suivants sur des données DEJA VUES : 

from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn import tree

... ainsi que votre 1-ppv maison

## Evaluation du classifieur

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn import tree

testedValue = valuesArray[30]
print("Tested Value is : " + str(labelsArray[30]))

mplClassifier = MLPClassifier().fit(valuesList, labelsList)
mplClassifier.predict([testedValue])

print(valuesArray.shape)
print(labelsArray.shape)


svmClassifier = svm.SVC().fit(valuesArray, labelsArray)
svmClassifier.predict([testedValue])


Afin d'évaluer le classifieur sur des données non vues, on a besoin de diviser les données initiales en 4 ensembles : 

- x_train et y_train : l'ensemble dit d'apprentissage, utilisé pour construire le classifieur (fit).
- x_test et y_test : l'ensemble dit de test, utilisé pour calculer la performance du classifieur sur des DONNEES NON VUES. 


### Exercice 4 Data split
Coder une fonction data_split(x, y, prob) permettant de diviser les données initiales (x et y) en  4 tableaux  x_train, x_test, y_train, y_test. (un tuple à renvoyer)
Le paramètre prob est un float compris entre 0 et 1 et désigne la probabilité qu'une ligne fasse partie, ou non, de l'ensemble d'apprentissage. Le ratio len(x_train)/len(x_test) sera donc sujet à variation au cours des appels.  

Indice Python : vous aurez besoin de zip et de l'opérateur de déballage (*)

In [ ]:
x=['a','b','c']
y=[1,2,3]
l=list(zip(x,y))
l

In [ ]:
list(zip(*l))

In [ ]:
#import random
#def data_split(x, y, prob):   

In [ ]:
badClassified = list()
def accuracy(y_pred,y) :
    res = []
    for i in range(len(y_pred)):
        if((y_pred[i] != y[i])):
            res.append(i)
    return res     
    #return (sum(y_pred == y))/len(y)

In [ ]:
#Usage :
#mplClassifier = MLPClassifier().fit(valuesArray, labelsArray)
#y_pred = mplClassifier.predict(valuesArray)
#print(accuracy(y_pred,labelsArray))

### Exercice 6 Performance du classifieurs OCR 
Mesurer le taux de bonnes réponses pour les différents classifieurs utilisés plus haut.  

### Exercice 7 Amélioration des performances
Ici, la stratégie est libre, seul compte le taux de bonnes réponses. 

On pourra toutefois se concentrer sur le prétraitement des images : 
- appliquer un filtre à l'image avec les fonctions d'ImageFilter
- changer la résolution avec resize
- la couleur (Mode L, LA, ...)
- techniques de centrage, de redressement, de lignes principales, ...

En machine learning, cette phase d'expérimentation est souvent déterminante.
Elle peut être motivée par : 
- la nature des exemples mal classés (y a t il du bruit, sont ils tordus, non centrés... ?)
- d'autres techniques OCR

Et rien ne vous empêche de : 
- tenter sur les caractères contrariés, ou de mélanger les 2
- déterminer les paramètres optimaux des classifieurs avec un gridSearch



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X_train, X_test, y_train, y_test = train_test_split(
    valuesArray, labelsArray, test_size=0.4, random_state=0)

X_train.shape, y_train.shape

X_test.shape, y_test.shape

mplClassifier = MLPClassifier().fit(valuesArray, labelsArray)
y_pred = mplClassifier.predict(X_test)
print(accuracy(y_pred,labelsArray))

clf = tree.DecisionTreeClassifier().fit(X_train, y_train)

clf.score(X_test, y_test) 

In [ ]:
from sklearn.model_selection import cross_val_score
clf = tree.DecisionTreeClassifier()
scores = cross_val_score(clf, valuesArray, labelsArray, cv=5)
scores 